<img align="right" src="files/logo_grupo_en_verde.png" style="width: 200px;"/> 
<br>
<br>
<br>
<br>
<img align="left" src="files/ee.png" style="width: 800px;"/>

In [7]:
# Importando e Inicializando a API do Google Engine + Iphython Display para visualizar os dados
import ee
ee.Initialize()

from IPython.display import Image

# Obtendo imagens Landsat

In [90]:
l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA_FMASK')

# Filtra a collection para o período ('2016-05-01', '2016-06-30')
filtered = l8.filterDate('2016-05-01', '2017-06-30');

# O redutor de mosaico seleciona o pixel mais recente nas áreas de sobreposição
l8filtrada = filtered.min()
Image(url=l8filtrada.getThumbUrl()) # Dessa forma não funciona

# Obtendo grid a ser processado

In [83]:
cenasLandsat = ee.FeatureCollection('ft:1cvotaqWbn-2QFJTl_pdfaeY3aoXGDf0rTFGzDsV5','geometry')

In [84]:
tile219069 = cenasLandsat.filter(ee.Filter.eq('TILE_T','T219069'))

In [85]:
gridGeometry = tile219069.geometry().bounds()

In [86]:
coordList = gridGeometry.coordinates().getInfo()

In [87]:
# Plotando os dados
Image(url=l8filtrada.getThumbUrl({'min':0, 'max':0.3, 'region':coordList, 'bands':'B4,B3,B2'}))

# Clipping

In [88]:
l8clip = l8filtrada.clip(tile219069.geometry())

In [89]:
Image(url=l8clip.getThumbUrl({'min':0, 'max':0.3, 'region':coordList, 'bands':'B4,B3,B2'}))

## Salvando dados no Google Drive

In [117]:
taskConfig = {
    "scale": 30,
    "maxPixels": 1.0E13,
    'description': 'Cena 219069',    
    "driveFolder": 'L8teste',
    'region': coordList
}

In [118]:
l8f = l8clip.multiply(10000).toInt16()
task = ee.batch.Export.image(l8f, 'cena219069', taskConfig)

In [119]:
task.start()

## Salvando dados no Earth Engine (Assets)

In [131]:
task = ee.batch.Export.image.toAsset(
  image = l8f,
  description = 'Cena 219069',
  assetId = 'users/fernandompimenta/Python/cena219069',
  scale = 30,
  region = coordList,
);

task.start()

# Utilizando o help